In [1]:
import pandas as pd
import numpy as np
from sklearn import metrics # for the evaluation
from sklearn.preprocessing import LabelEncoder,MinMaxScaler
import tensorflow as tf 

In [2]:
df = pd.read_csv("data/mongo_data.csv", index_col='ts')
df = df.drop('Unnamed: 0', 1)
#Remove cols with the same value

def unique_cols(df):
    a = df.to_numpy() # df.values (pandas<0.24)
    return (a[0] == a).all(0)

df = df.loc[:,np.invert(unique_cols(df))]
print(df.shape)
df.tail()

C:\Users\pedro\AppData\Local\Temp\ipykernel_43488\468073868.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df = df.drop('Unnamed: 0', 1)


(467334, 39)


,S_SUM,U_L1_L2,I_SUM,F,U_L3_L1,C_phi_L1,U_L3_N,RealEc_L1,RealEc_L2,RealEc_L3,...,S_L3,Q_L1,S_L2,U_L2_N,S_L1,U_L1_N,I_L1,RealEc_SUM,I_L2,I_L3
ts,,,,,,,,,,,,,,,,,,,,,
2022-02-22T11:17:24.127703575Z,123259.65,411.39,1.12,50.01,411.12,0.99,236.57,31345348,30444914,30360338,...,42346.98,6085.66,39469.35,238.10,41443.33,237.23,174.70,92150704,165.77,179.00
2022-02-22T11:17:25.439348217Z,122868.31,411.33,1.05,50.01,409.30,0.98,236.51,31345362,30444928,30360352,...,40454.14,7272.30,39607.47,237.89,42806.71,236.40,181.08,92150752,166.49,171.04
2022-02-22T11:17:26.705083973Z,123108.94,411.02,1.04,50.01,410.57,0.98,237.13,31345374,30444940,30360364,...,40749.97,8029.02,41030.93,237.73,41328.03,237.20,174.23,92150784,172.60,171.85
2022-02-22T11:17:28.014494747Z,123048.30,410.90,1.02,50.01,410.75,0.99,236.20,31345388,30444952,30360380,...,42542.77,5632.11,39168.28,237.67,41337.26,237.06,174.37,92150824,164.80,180.11
2022-02-22T11:17:29.306443661Z,122208.24,409.81,1.08,50.01,409.76,0.99,235.70,31345400,30444964,30360392,...,41760.77,5949.70,39547.37,237.24,40900.10,236.52,172.93,92150864,166.70,177.18


In [3]:
# convert series to supervised learning
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
	n_vars = 1 if type(data) is list else data.shape[1]
	df = pd.DataFrame(data)
	cols, names = list(), list()
	# input sequence (t-n, ... t-1)
	for i in range(n_in, 0, -1):
		cols.append(df.shift(i))
		names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
	# forecast sequence (t, t+1, ... t+n)
	for i in range(0, n_out):
		cols.append(df.shift(-i))
		if i == 0:
			names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
		else:
			names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
	# put it all together
	agg = pd.concat(cols, axis=1)
	agg.columns = names
	# drop rows with NaN values
	if dropnan:
		agg.dropna(inplace=True)
	return agg

In [4]:
scaler = MinMaxScaler()
d = scaler.fit_transform(df)
scaled_df = pd.DataFrame(d, columns=df.columns, index=df.index)
scaled_df.head()

,S_SUM,U_L1_L2,I_SUM,F,U_L3_L1,C_phi_L1,U_L3_N,RealEc_L1,RealEc_L2,RealEc_L3,...,S_L3,Q_L1,S_L2,U_L2_N,S_L1,U_L1_N,I_L1,RealEc_SUM,I_L2,I_L3
ts,,,,,,,,,,,,,,,,,,,,,
2022-02-15T10:24:58.903000343Z,0.743565,0.525206,0.537234,0.681818,0.446846,0.98,0.519724,0.000000,0.000000,0.000000,...,0.707483,0.627333,0.739096,0.498763,0.747683,0.466906,0.756375,0.000000,0.752907,0.711968
2022-02-15T10:24:59.374508444Z,0.739004,0.493876,0.659574,0.681818,0.443589,0.99,0.453649,0.000003,0.000003,0.000003,...,0.737740,0.507947,0.731334,0.479960,0.712904,0.448435,0.722294,0.000003,0.746200,0.746613
2022-02-15T10:25:00.655871379Z,0.741547,0.513814,0.452128,0.681818,0.499260,0.99,0.510848,0.000006,0.000006,0.000006,...,0.725338,0.558912,0.735819,0.502721,0.727841,0.507953,0.733805,0.000006,0.749335,0.730465
2022-02-15T10:25:01.887888454Z,0.743124,0.527770,0.531915,0.681818,0.469944,0.99,0.522682,0.000009,0.000009,0.000009,...,0.739171,0.497014,0.736202,0.503711,0.718695,0.485890,0.725887,0.000009,0.749663,0.743651
2022-02-15T10:25:03.145591485Z,0.739277,0.455711,0.659574,0.681818,0.444773,0.99,0.449211,0.000012,0.000012,0.000012,...,0.744462,0.509748,0.740470,0.445819,0.698377,0.455105,0.707189,0.000012,0.757755,0.753730


In [5]:
from sklearn.decomposition import PCA
pca = PCA(n_components=0.99)

ds = pca.fit_transform(scaled_df)
print(pca.n_components_)

pcn = pca.n_components_

6


In [7]:
history_window = 60 #seconds
prediction_window = 1
# frame as supervised learning
reframed = series_to_supervised(ds, history_window, prediction_window)
# drop columns we don't want to predict
print(reframed.head())

    var1(t-60)  var2(t-60)  var3(t-60)  var4(t-60)  var5(t-60)  var6(t-60)  \
60   -1.734031    1.138734    0.156246   -0.120754   -0.133430    0.173092   
61   -1.737445    1.129785    0.246315   -0.149495   -0.124952   -0.073792   
62   -1.717033    1.147932    0.127730   -0.083905   -0.138044    0.070109   
63   -1.726197    1.137969    0.138930   -0.114588   -0.131646   -0.015550   
64   -1.738183    1.128171    0.272559   -0.153964   -0.126794   -0.147222   

    var1(t-59)  var2(t-59)  var3(t-59)  var4(t-59)  ...  var3(t-1)  var4(t-1)  \
60   -1.737445    1.129785    0.246315   -0.149495  ...  -0.355542  -0.159094   
61   -1.717033    1.147932    0.127730   -0.083905  ...  -0.290775  -0.168899   
62   -1.726197    1.137969    0.138930   -0.114588  ...  -0.431674  -0.146096   
63   -1.738183    1.128171    0.272559   -0.153964  ...  -0.322037  -0.212285   
64   -1.728246    1.127978   -0.319102   -0.171580  ...  -0.282756  -0.176592   

    var5(t-1)  var6(t-1)   var1(t)   var2(t)

In [13]:
values = reframed.values
len_values = values.shape[0]
# split into train and test sets 
n_train_seconds = int(0.7*len_values) #70% dos valores
n_test_seconds =  int(0.9*len_values) #20% dos valores
train = values[:n_train_seconds, :]
test = values[n_train_seconds:n_test_seconds, :]
# split into input and outputs
train_X, train_y = train[:, :-pcn], train[:, -pcn:]
test_X, test_y = test[:, :-pcn], test[:, -pcn:]
# reshape input to be 3D [samples, timesteps, features]
train_X = train_X.reshape((train_X.shape[0], 1, train_X.shape[1]))
test_X = test_X.reshape((test_X.shape[0], 1, test_X.shape[1]))
print(train_X.shape, train_y.shape, test_X.shape, test_y.shape)

(327091, 1, 360) (327091, 6) (93455, 1, 360) (93455, 6)


In [6]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import GRU

# design network
model = Sequential()
model.add(GRU(32, input_shape=(train_X.shape[1], train_X.shape[2])))
model.add(Dense(units = 512, activation = 'relu'))
model.add(Dense(pcn, activation='linear'))
model.compile(loss='mean_squared_error', optimizer='adam')
# fit network
history = model.fit(train_X, train_y, epochs=300, batch_size=144, validation_data=(test_X, test_y), verbose=2, shuffle=False)

NameError: name 'train_X' is not defined

In [ ]:
history_results = pd.DataFrame(list(zip(history.history['loss'], history.history['val_loss'])), columns=['Loss', 'Validation Loss'])
history_results.to_csv('results/PCA_GRU_60secs_MSE_loss.csv')
model.save('models/PCA_GRU_60secs_MSE.h5')

In [ ]:
from sklearn.metrics import mean_squared_error

#Test for the day after
n_rtest_seconds =  int(0.1*24*60*60) #10%
rtest = values[-n_rtest_seconds:, :]

rtest_X, rtest_y = rtest[:, :-pcn], rtest[:, -pcn:]
rtest_X = rtest_X.reshape((rtest_X.shape[0], 1, rtest_X.shape[1]))
# make a prediction
yhat = model.predict(rtest_X)

In [ ]:
mu = np.mean(scaled_df, axis=0)

Xhat = np.dot(yhat, pca.components_[:pcn,:])
Xhat_pca = np.array([Xhat[i] + mu for i in range(len(Xhat))])

In [ ]:
prediction_results = pd.DataFrame(np.transpose(Xhat_pca))
prediction_results.to_csv('results/PCA_GRU_60secs_MSE_prediction.csv')